## Analyze Amazon Review Sentiment Analysis Using an LLM

In [19]:
# Import Statements
import pandas as pd
import os
import pickle
import random

from azure.storage.blob import BlobServiceClient
import gzip
import json

from dotenv import load_dotenv
import openai

In [2]:
# Import Review Dataframe
r_df = pd.read_csv("reviews.csv")

# Import Product Metadata
m_df = pd.read_csv("metadata.csv")

C:\Users\samir\AppData\Local\Temp\ipykernel_3556\3701032164.py:2: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  r_df = pd.read_csv("reviews.csv")
C:\Users\samir\AppData\Local\Temp\ipykernel_3556\3701032164.py:5: DtypeWarning: Columns (9,16) have mixed types. Specify dtype option on import or set low_memory=False.
  m_df = pd.read_csv("metadata.csv")


In [3]:
# First, we calculate the average rating for each product
average_ratings = r_df.groupby('asin')['overall'].mean().reset_index()

# Then, we rename the column from 'overall' to 'average_rating'
average_ratings.rename(columns={'overall': 'average_rating'}, inplace=True)

# Now, we merge the metadata DataFrame with the average ratings DataFrame using 'asin' as the key
merged_df = pd.merge(m_df, average_ratings, on='asin', how='inner')

In [4]:
# Print unique 'asin' in m_df
print("Unique ASINs in metadata: ", m_df['asin'].nunique())

# Print unique 'asin' in average_ratings
print("Unique ASINs in average ratings: ", average_ratings['asin'].nunique())

# Find common 'asin' between m_df and average_ratings
common_asins = set(m_df['asin']).intersection(set(average_ratings['asin']))
print("Number of common ASINs: ", len(common_asins))

Unique ASINs in metadata:  293172
Unique ASINs in average ratings:  150907
Number of common ASINs:  7


In [5]:
# Get the 'asin' values from merged_df
merged_asins = merged_df['asin']

# Keep only the rows in r_df where 'asin' is in merged_asins
filtered_r_df = r_df[r_df['asin'].isin(merged_asins)]

In [6]:
# Count the occurrences of each 'asin' in filtered_r_df
asin_counts = filtered_r_df['asin'].value_counts()

# Print the counts
print(asin_counts)

asin
B0007RAE7A    338
B00EVWFNE6    194
B00EZIC2E0     21
B00EVRG3EK     11
B00EQ5I3BS      8
B00EZEZFSE      3
B00EOAZJQW      1
Name: count, dtype: int64


In [7]:
# Find counts of ASINs
asin_counts_df = r_df.groupby('asin').size().reset_index(name='count')
asin_counts_df

,asin,count
0,0003709469,1
1,0006240682,6
2,0006481531,1
3,0007150725,2
4,0007243391,1
...,...,...
150902,B01AHGKHEU,10
150903,B01AHGLNCA,10
150904,B01AHGNNJ6,10
150905,B01AHGNPW6,10


In [8]:
# Filter only ASINs that show up more than 3 times
filtered_asin_df = asin_counts_df[asin_counts_df['count'] >= 3].reset_index(drop=True)

# Create List of unique ASIN codes
asin_list = filtered_asin_df['asin'].tolist()

In [9]:
filtered_asin_df

,asin,count
0,0006240682,6
1,0007580274,6
2,0008193681,3
3,0020126123,5
4,0021602301,3
...,...,...
79246,B01AHGJ9YE,10
79247,B01AHGKHEU,10
79248,B01AHGLNCA,10
79249,B01AHGNNJ6,10


In [10]:
# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
storage_account = os.environ.get("STORAGE_ACCOUNT")
container = os.environ.get("CONTAINER")
sas_token = os.environ.get("ADLS_SAS_TOKEN")
sas_url = os.environ.get("SAS_URL")

account_url = f"https://{storage_account}.blob.core.windows.net"
service_client = BlobServiceClient(account_url=account_url, credential=sas_token)

# Access the container
container_client = service_client.get_container_client(container)

In [ ]:
pickle_in = open("amazon_metadata.pickle","rb")
amazon_metadata = pickle.load(pickle_in)

# Extract unique ASINs from the filtered dataframe
#asin_list = filtered_df['asin'].unique()

# Initialize an empty DataFrame
valid_products_df = pd.DataFrame()

# Loop through the first 30 items in the list
for i in range(len(amazon_metadata)-1):
    # Get blob client for the i-th blob
    blob_client = container_client.get_blob_client(amazon_metadata[i])

    # Download the blob and decode it
    downloaded_blob = blob_client.download_blob().readall()
    json_data = gzip.decompress(downloaded_blob).decode('utf-8')

    # Read JSON data into a DataFrame
    temp_df = pd.read_json(json_data, lines=True)

    # Only keep rows in temp_df where the asin is in asin_list
    temp_df = temp_df[temp_df['asin'].isin(asin_list)]

    # Append the data to the main DataFrame
    valid_products_df = pd.concat([valid_products_df, temp_df], ignore_index=True)

In [13]:
valid_products_df

,also_buy,also_view,asin,brand,category,date,description,details,feature,fit,image,main_cat,price,rank,similar_item,tech1,tech2,title
0,"[B075JYWSTM, B07BMHRCGD, B01B48NSRI, B06WLNKP1...","[B07BMHRCGD, B075JYWSTM, B07BXDRZR3, B075JQQ4F...",B0007RAE7A,,"[Clothing, Shoes & Jewelry, Men, Clothing, Shi...","<div class=""a-fixed-left-grid a-spacing-none"">...","[For nearly 70 years, Wrangler has staked its ...",{},"[100% Cotton, Imported, Machine Wash, Button-f...","class=""a-normal a-align-center a-spacing-smal...",[https://images-na.ssl-images-amazon.com/image...,"<img src=""https://images-na.ssl-images-amazon....",$15.99 - $17.87,"9,309 in Clothing, Shoes & Jewelry (",,,,Wrangler Authentics Men's Short-Sleeve Classic...
1,[],[],B00EQ5I3BS,,"[Clothing, Shoes & Jewelry, Women, Shoes, Athl...","<div class=""a-fixed-left-grid a-spacing-none"">...","[Nike Women's Wmns Free 3.0 Flyknit, PURE PLAT...",{},"[synthetic-and-fabric, Rubber sole, Model Numb...",,[https://images-na.ssl-images-amazon.com/image...,"<img src=""https://images-na.ssl-images-amazon....",,[],,,,"NIKE Women's WMNS Free Flyknit 3.0, Black/Whit..."
2,[],[],B00EVRG3EK,White Sierra,"[Clothing, Shoes & Jewelry, Novelty & More, Cl...",,[Insect Shield is an odorless invisible insect...,"{'ASIN: ': 'B00EVRG3EK', 'Domestic Shipping: '...","[100% Polyester Mesh Knit, Imported, Insect Sh...",,[https://images-na.ssl-images-amazon.com/image...,Sports & Outdoors,,"652,572 in Sports & Outdoors (",,,,White Sierra Women's Bug Free Zip Hoody
3,"[B07DHGZWP3, B07FV2M36L, B07DLRWVDV, B07G9W6LP...","[B07DHGZWP3, B07BNVSS2H, B07G9W6LPC, B07FV2M36...",B00EVWFNE6,,"[Clothing, Shoes & Jewelry, Girls, Shoes, Athl...","<div class=""a-fixed-left-grid a-spacing-none"">...",[Water shoe for all summer outdoor fun with bu...,{},"[100% Manmade, Synthetic sole, Water shoe feat...","class=""a-normal a-align-center a-spacing-smal...",[https://images-na.ssl-images-amazon.com/image...,"<img src=""https://images-na.ssl-images-amazon....",$6.44 - $27.97,"73,637 in Clothing, Shoes & Jewelry (",,,,Northside Kids' Brille
4,[B071HC7RPG],"[B071HC7RPG, B07BTD15YV]",B00EZEZFSE,boxercraft,"[Clothing, Shoes & Jewelry, Women, Clothing, A...","<div class=""a-fixed-left-grid a-spacing-none"">...","[- 2.4 oz., 100% moisture wicking polyester - ...",{},"[100% moisture-wicking polyester., Built-in po...",,[https://images-na.ssl-images-amazon.com/image...,Sports & Outdoors,$8.99 - $21.22,"2,142,906 in Clothing, Shoes & Jewelry (",,,,Boxercraft P62 - Ladies' Novelty Velocity Runn...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79763,[],"[B07K2TGSJS, B007RKG8TS, B07HZSS31B, B005BW8DT...",B00APNC7D0,,"[Clothing, Shoes & Jewelry, Men, Surf, Skate &...","<div class=""a-fixed-left-grid a-spacing-none"">...",[The Union Hi TX remixes traditional Union hi-...,{},"[synthetic-and-rubber, Cupsole Shoe, You can r...","class=""a-normal a-align-center a-spacing-smal...",[],"<img src=""https://images-na.ssl-images-amazon....",$62.95,"3,048,517 in Clothing, Shoes & Jewelry (",,,,DC Men's Union High TX Lace-Up Fashion Sneaker
79764,[B073HKKD19],"[B07C17K6T7, B01MQCJAXR, B015RW0HGC, B073CXJVR...",B00AVWO9PE,So Sexy Lingerie,"[Clothing, Shoes & Jewelry, Women, Clothing, L...","<div class=""a-fixed-left-grid a-spacing-none"">...",[Fits-like-a-glove stretch satin booty short f...,{},"[Authentic SO SEXY LINGERIE (TM) brand, Made o...","class=""a-normal a-align-center a-spacing-smal...",[https://images-na.ssl-images-amazon.com/image...,"<img src=""https://images-na.ssl-images-amazon....",$14.99 - $18.99,"1,490,743 in Clothing, Shoes & Jewelry (",,,,So Sexy Lingerie Booty Short Panties Body Hugg...
79765,[B01E04X8K0],[],B00AWO5WBG,,"[Clothing, Shoes & Jewelry, Women, Shoes, Pump...","<div class=""a-fixed-left-grid a-spacing-none"">...",[<li>Classy open toe strappy wedding shoes.com...,{},"[satin, Imported, Rubber sole, 3.5"" navy blue ...",,[https://images-na.ssl-images-amazon.com/image...,"<img src=""https://images-

In [22]:
# Select a random ASIN to analyze

r = random.randint(0, len(valid_products_df))
ASIN_TO_ANALYZE = valid_products_df.at[r,'asin']
analyze_df = r_df[r_df['asin'] == ASIN_TO_ANALYZE].reset_index(drop=True)

In [23]:
# Display the Random ASIN product details

product_details = valid_products_df[valid_products_df['asin'] == ASIN_TO_ANALYZE]
product_details

,also_buy,also_view,asin,brand,category,date,description,details,feature,fit,image,main_cat,price,rank,similar_item,tech1,tech2,title
19940,[],[],B017IY9KYW,Seasonstorm,"[Arts, Crafts & Sewing, Scrapbooking & Stampin...",,[Description: <br> 100% Brand New and High Qua...,{},"[Size: About 95x150 mm (3.75x5.9 inches), Funn...",,[https://images-na.ssl-images-amazon.com/image...,"Arts, Crafts & Sewing",,"["">#331,546 in Arts, Crafts & Sewing (See Top ...",,,,Seasonstorm (TM) 6 sheets/set Cute Simple Life...


In [24]:
# Load environment variables from .env file
load_dotenv()

# Retrieve Open AI API key
api_key = os.environ.get("OPEN_AI_API_KEY")
openai.api_key = api_key

In [25]:
# Analyze the sentiment of the reviews

analysis_template = """
Please analyze all of these reviews for this product and find some common themes about what the 
customers are saying about the product. Each review will be seperated by a '|' character 
Please return the most common themes in order of most to least relevant.
"""

# Consolidate Reviews in long string
all_reviews = '|'.join(analyze_df['reviewText'])

messages=[
    {"role": "system", "content": analysis_template},
    {"role": "user", "content": all_reviews}
]

response = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=messages,
  max_tokens=3500,
  temperature=0,
)

sentiment = response['choices'][0]['message']['content'].strip()
print(sentiment)

Common themes:
1. Quality issues: Several customers mentioned that the stickers were hard to peel off the backing without ripping them, and some mentioned that the cuts on the stickers were off, requiring them to trim them by hand.
2. Size discrepancy: One customer mentioned that the stickers were smaller than advertised.
3. Cute and pretty: Many customers found the stickers to be cute and pretty.
4. Usefulness: Some customers mentioned using the stickers in their planners and finding them useful for brightening up the pages.
5. Variety: One customer mentioned that there was a great variety of stickers.
6. Customer service: One customer mentioned having a positive experience with customer service.
7. Timely delivery: One customer mentioned that the stickers arrived well before the estimated date.
8. Plain design: One customer mentioned that the stickers were a little plain.
9. Gift option: One customer mentioned giving the stickers to their students as treats.

Order of relevance:
1. Q